<a href="https://colab.research.google.com/github/sakiosa/gcp/blob/main/Copy_of_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies (Colab)
!pip install tensorflow keras scikit-learn pandas numpy matplotlib transformers

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import urllib.request
import gzip

# 1. Load Dataset
url = "https://snap.stanford.edu/data/finefoods.txt.gz"  # Kaggle mirror dataset

# Read the gzipped file from the URL
data = []
with urllib.request.urlopen(url) as response:
    with gzip.GzipFile(fileobj=response) as f:
        entry = {}
        for line in f:
            line = line.decode('latin-1').strip() # Decode and strip whitespace
            if line:
                # Split only if the line contains ': '
                if ': ' in line:
                    key, value = line.split(': ', 1)
                    entry[key.replace('/', '_')] = value # Replace '/' with '_' for valid column names
                # If line doesn't contain ': ', it might be a separator or part of the text
                # We can choose to ignore it or handle it differently
                # For now, we'll ignore lines that don't fit the key: value pattern
            else:
                if entry:
                    data.append(entry)
                entry = {}
        if entry: # Append the last entry
            data.append(entry)

df = pd.DataFrame(data)

# Select relevant columns and handle missing values
df = df[['review_text', 'review_score']].dropna()
df['review_score'] = pd.to_numeric(df['review_score'], errors='coerce')
df.dropna(subset=['review_score'], inplace=True)
df['label'] = (df['review_score'].astype(float) >= 4).astype(int)  # Positive (1) if >=4 stars, else Negative (0)

# 2. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(df['review_text'], df['label'], test_size=0.2, random_state=42)

# 3. Tokenization
max_words = 20000
max_len = 200
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_len)
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_len)

# 4. Model (BiLSTM)
model = Sequential([
    Embedding(max_words, 128, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train_seq, y_train, validation_data=(X_test_seq, y_test), epochs=3, batch_size=64)

# 5. Evaluation
y_pred = (model.predict(X_test_seq) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/3
 164/7106 ━━━━━━━━━━━━━━━━━━━━ 1:18:56 682ms/step - accuracy: 0.7863 - loss: 0.5263

In [ ]:
# ============================================================
# 0) SETUP
# ============================================================
!pip -q install numpy pandas scikit-learn tensorflow==2.15.0 matplotlib seaborn streamlit -U

import os, math, random
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("TF:", tf.__version__)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.9 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement tensorflow==2.15.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.15.0
TF: 2.19.0


In [ ]:
# ============================================================
# 1) SYNTHETIC DATA GENERATION
# ============================================================
np.random.seed(42)

WEEKS = 156  # 3 years
N_STORES = 150
N_PRODUCTS = 400
N_REGIONS = 5
N_CATEGORIES = 6

start_date = pd.to_datetime("2021-01-03")  # a Sunday
calendar = pd.DataFrame({"week_idx": np.arange(WEEKS)})
calendar["date"] = calendar["week_idx"].apply(lambda k: start_date + pd.to_timedelta(7*k, unit="D"))
calendar["weekofyear"] = calendar["date"].dt.isocalendar().week.astype(int)
calendar["year"] = calendar["date"].dt.year

# simple holiday spikes (e.g., weeks 47-52)
calendar["is_holiday_peak"] = calendar["weekofyear"].between(47, 52).astype(int)
# seasonality terms
calendar["sin52"] = np.sin(2*np.pi*calendar["weekofyear"]/52)
calendar["cos52"] = np.cos(2*np.pi*calendar["weekofyear"]/52)

# Meta tables
stores = pd.DataFrame({
    "store_id": np.arange(N_STORES),
    "region_id": np.random.randint(0, N_REGIONS, size=N_STORES),
    "store_size": np.random.choice([1,2,3], size=N_STORES, p=[0.3,0.5,0.2])  # 1=small,2=med,3=large
})
products = pd.DataFrame({
    "product_id": np.arange(N_PRODUCTS),
    "category_id": np.random.randint(0, N_CATEGORIES, size=N_PRODUCTS),
    "base_price": np.round(np.random.uniform(3, 80, size=N_PRODUCTS),2),
    "elasticity": np.random.uniform(0.3, 1.2, size=N_PRODUCTS)  # price elasticity coef
})

# Cross-join skeleton: to keep memory reasonable, sample subset
pairs = pd.MultiIndex.from_product([stores["store_id"], products["product_id"]], names=["store_id","product_id"])
pairs = pairs.to_frame(index=False)
# To speed up demo, choose a subset (e.g., 60 stores x 200 products)
pairs = pairs.sample(n=60*200, random_state=42).reset_index(drop=True)

def simulate_temp(region_id, week_idx):
    # warmer in weeks 20-35 (northern summer), vary by region
    base = 10 + 12*np.sin(2*np.pi*(week_idx-20)/52.0) + region_id*0.8
    noise = np.random.normal(0, 1.5)
    return base + noise

rows = []
for _, r in pairs.iterrows():
    store_id = r["store_id"]
    product_id = r["product_id"]
    region_id = stores.loc[stores.store_id==store_id, "region_id"].values[0]
    store_size = stores.loc[stores.store_id==store_id, "store_size"].values[0]
    base_price = products.loc[products.product_id==product_id, "base_price"].values[0]
    elasticity = products.loc[products.product_id==product_id, "elasticity"].values[0]
    category_id = products.loc[products.product_id==product_id, "category_id"].values[0]

    # baseline demand by category & store size
    cat_base = {0:6,1:8,2:10,3:12,4:9,5:7}[category_id]
    store_scale = {1:0.7,2:1.0,3:1.3}[store_size]
    trend = np.random.uniform(-0.003, 0.003)  # slight up/down trend

    # random weeks of promotions
    promo_weeks = set(np.random.choice(range(WEEKS), size=int(WEEKS*0.25), replace=False))

    for k in range(WEEKS):
        w = calendar.iloc[k]
        # exogenous
        temp = simulate_temp(region_id, k)
        holiday = w["is_holiday_peak"]
        sine, cosine = w["sin52"], w["cos52"]

        # dynamic price with promos
        promo = 1 if k in promo_weeks else 0
        discount = np.random.choice([0,0.1,0.2,0.3], p=[0.75,0.15,0.08,0.02]) if promo else 0
        price = round(base_price*(1-discount), 2)

        # demand drivers
        seasonality = 1.0 + 0.35*sine + 0.15*cosine
        holiday_boost = 1.25 if holiday else 1.0
        temp_effect = 1 + 0.03*np.tanh((temp-15)/10.0) if category_id in [2,3] else 1.0  # apparel sensitive to weather
        price_effect = (1 + elasticity*(base_price-price)/base_price)

        # multiplicative model
        mean_sales = cat_base*store_scale*seasonality*holiday_boost*temp_effect*price_effect*(1 + trend*k)
        noise = np.random.lognormal(mean=0, sigma=0.25)
        units = max(0.0, np.random.poisson(lam=max(0.1, mean_sales))*noise)

        rows.append([w["date"], k, store_id, region_id, store_size, product_id, category_id,
                     price, discount, promo, temp, holiday, sine, cosine, units])

df = pd.DataFrame(rows, columns=[
    "date","week_idx","store_id","region_id","store_size","product_id","category_id",
    "price","discount","promo","temp","is_holiday","sin52","cos52","units"
])

print("Rows:", len(df), "Sample:")
df.head()


Rows: 1872000 Sample:


,date,week_idx,store_id,region_id,store_size,product_id,category_id,price,discount,promo,temp,is_holiday,sin52,cos52,units
0,2021-01-03,0,31,3,2,228,4,14.68,0.0,0,5.747630,0,0.120537,0.992709,19.242739
1,2021-01-10,1,31,3,2,228,4,14.68,0.0,0,7.022994,0,0.120537,0.992709,14.785478
2,2021-01-17,2,31,3,2,228,4,14.68,0.0,0,4.182483,0,0.239316,0.970942,10.763913
3,2021-01-24,3,31,3,2,228,4,14.68,0.0,0,4.224675,0,0.354605,0.935016,4.504933
4,2021-01-31,4,31,3,2,228,4,14.68,0.0,0,1.083598,0,0.464723,0.885456,13.027953


In [ ]:
# ============================================================
# 2) SUPERVISED DATASET: windowed sequences for LSTM/TCN
#    Past T weeks -> Forecast next H weeks
# ============================================================
T = 12   # lookback
H = 4    # horizon (predict next 4 weeks)
key_cols = ["store_id","product_id"]
sort_cols = ["store_id","product_id","week_idx"]

df = df.sort_values(sort_cols).reset_index(drop=True)

# Build lagged features
def add_lags(group):
    group = group.sort_values("week_idx")
    for lag in [1,2,3,4,8,12]:
        group[f"units_lag{lag}"] = group["units"].shift(lag)
    group["units_ma4"] = group["units"].rolling(4).mean().shift(1)
    group["promo_last4"] = group["promo"].rolling(4).sum().shift(1)
    return group

df = df.groupby(key_cols, group_keys=False).apply(add_lags)
df = df.dropna().reset_index(drop=True)

# numeric & categorical features
num_feats = ["price","discount","temp","sin52","cos52","units_lag1","units_lag2","units_lag3","units_lag4","units_ma4","promo_last4"]
cat_feats = ["store_id","product_id","region_id","category_id","store_size","promo","is_holiday"]

feat_cols = num_feats + cat_feats

# Create sequences per (store, product)
X_seq, X_cat_seq, y_seq = [], [], []
groups = df.groupby(key_cols)
for (sid, pid), g in groups:
    g = g.sort_values("week_idx")
    # we need T past steps and H targets ahead
    for i in range(T, len(g)-H+1):
        past = g.iloc[i-T:i]
        future = g.iloc[i:i+H]
        if (past["week_idx"].max()+H) != future["week_idx"].max():
            # ensure continuity
            continue
        X_seq.append(past[num_feats].values)                 # (T, num_feats)
        X_cat_seq.append(past[cat_feats].values.astype(int)) # (T, cat_feats)
        y_seq.append(future["units"].values)                 # (H,)

X_num = np.array(X_seq, dtype=np.float32)
X_cat = np.array(X_cat_seq, dtype=np.int32)
y = np.array(y_seq, dtype=np.float32)

print("Shapes:", X_num.shape, X_cat.shape, y.shape)


/tmp/ipython-input-494742477.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(key_cols, group_keys=False).apply(add_lags)


Shapes: (1548000, 12, 11) (1548000, 12, 7) (1548000, 4)


In [ ]:
# ============================================================
# 3) SPLIT & SCALE
# ============================================================
N = len(X_num)
# split by index ~ time order; user may choose cutoffs by date
train_ratio, valid_ratio = 0.7, 0.15
n_train = int(N*train_ratio)
n_valid = int(N*(train_ratio+valid_ratio))

Xn_train, Xn_valid, Xn_test = X_num[:n_train], X_num[n_train:n_valid], X_num[n_valid:]
Xc_train, Xc_valid, Xc_test = X_cat[:n_train], X_cat[n_train:n_valid], X_cat[n_valid:]
y_train, y_valid, y_test = y[:n_train], y[n_train:n_valid], y[n_valid:]

# scale only numeric channels across T steps (flatten -> fit -> reshape)
scaler = StandardScaler()
Xn_train_flat = Xn_train.reshape(-1, Xn_train.shape[-1])
scaler.fit(Xn_train_flat)
def scale_num(Xn):
    flat = Xn.reshape(-1, Xn.shape[-1])
    flat = scaler.transform(flat)
    return flat.reshape(Xn.shape)
Xn_train = scale_num(Xn_train)
Xn_valid = scale_num(Xn_valid)
Xn_test  = scale_num(Xn_test)

print("Train/Valid/Test:", Xn_train.shape, Xn_valid.shape, Xn_test.shape)


Train/Valid/Test: (1083600, 12, 11) (232200, 12, 11) (232200, 12, 11)


In [ ]:
# ============================================================
# 4) MODEL: Embeddings + Temporal layers -> H outputs
# ============================================================
tf.keras.backend.clear_session()

# Embedding sizes
store_vocab, prod_vocab = N_STORES, N_PRODUCTS
region_vocab, cat_vocab, size_vocab = N_REGIONS, N_CATEGORIES, 4
promo_vocab, holiday_vocab = 2, 2

def embedding_block(inp, vocab, dim):
    emb = layers.Embedding(input_dim=vocab, output_dim=dim, mask_zero=False)
    return emb(inp)

# Inputs
inp_num = keras.Input(shape=(T, len(num_feats)), name="num")
inp_cat = keras.Input(shape=(T, len(cat_feats)), name="cat", dtype="int32")

# Slice categorical columns by fixed order:
idx = {name:i for i,name in enumerate(cat_feats)}
cat_store = layers.Lambda(lambda z: z[:,:,idx["store_id"]])(inp_cat)
cat_prod  = layers.Lambda(lambda z: z[:,:,idx["product_id"]])(inp_cat)
cat_region= layers.Lambda(lambda z: z[:,:,idx["region_id"]])(inp_cat)
cat_cat   = layers.Lambda(lambda z: z[:,:,idx["category_id"]])(inp_cat)
cat_size  = layers.Lambda(lambda z: z[:,:,idx["store_size"]])(inp_cat)
cat_promo = layers.Lambda(lambda z: z[:,:,idx["promo"]])(inp_cat)
cat_hol   = layers.Lambda(lambda z: z[:,:,idx["is_holiday"]])(inp_cat)

# Embeddings (time-distributed IDs)
e_store = embedding_block(cat_store, store_vocab, 8)
e_prod  = embedding_block(cat_prod,  prod_vocab, 8)
e_region= embedding_block(cat_region,region_vocab, 4)
e_cat   = embedding_block(cat_cat,   cat_vocab,   4)
e_size  = embedding_block(cat_size,  size_vocab,  3)
e_promo = embedding_block(cat_promo, promo_vocab, 2)
e_hol   = embedding_block(cat_hol,   holiday_vocab, 2)

# Concatenate categorical embeddings along last dim
emb_cat = layers.Concatenate(axis=-1)([e_store,e_prod,e_region,e_cat,e_size,e_promo,e_hol])

# Merge numeric + embeddings
x = layers.Concatenate(axis=-1)([inp_num, emb_cat])

# Temporal stack: 1D Conv (TCN-like) -> BiLSTM
x = layers.Conv1D(64, kernel_size=3, padding="causal", activation="relu")(x)
x = layers.Conv1D(64, kernel_size=3, padding="causal", activation="relu")(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=False))(x)
x = layers.Dropout(0.25)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.2)(x)

out = layers.Dense(H, activation="relu")(x)  # predict next H weeks of units

model = keras.Model(inputs=[inp_num, inp_cat], outputs=out)
model.compile(optimizer=keras.optimizers.Adam(1e-3), loss="mse",
              metrics=[keras.metrics.MeanAbsoluteError(name="mae")])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ cat (InputLayer)    │ (None, 12, 7)     │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 12)        │          0 │ cat[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 12)        │          0 │ cat[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 12)        │          0 │ cat[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_3 (Lambda)   │ (None, 12)        │          0 │ cat[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_4 (Lambda)   │ (None, 12)        │          0 │ cat[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_5 (Lambda)   │ (None, 12)        │          0 │ cat[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_6 (Lambda)   │ (None, 12)        │          0 │ cat[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 12, 8)     │      1,200 │ lambda[0][0]      │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 12, 8)     │      3,200 │ lambda_1[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 12, 4)     │         20 │ lambda_2[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 12, 4)     │         24 │ lambda_3[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 12, 3)     │         12 │ lambda_4[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, 12, 2)     │          4 │ lambda_5[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_6         │ (None, 12, 2)     │          4 │ lambda_6[0][0]    │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ num (InputLayer)    │ (None, 12, 11)    │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 12, 31)    │          0 │ embedding[0][0],  │
│ (Concatenate)       │                   │            │ embedding_1[0][0… │
│                     │                   │            │ embedding_2[0][0… │
│                     │                   │            │ embedding_3[0][0… │
│                     │                   │            │ embedding_4[0][0… │
│                     │                   │            │ embedding_5[0][0… │
│                     │                   │            │ embedding_6[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 12, 42)    │          0 │ num[0][0],        │
│ (Concatenate)       │                   │            │ concatenate[0][0

 Total params: 108,020 (421.95 KB)

 Trainable params: 108,020 (421.95 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ============================================================
# 5) TRAINING
# ============================================================
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-5),
]

hist = model.fit(
    x={"num":Xn_train, "cat":Xc_train},
    y=y_train,
    validation_data=({"num":Xn_valid,"cat":Xc_valid}, y_valid),
    epochs=30,
    batch_size=512,
    callbacks=callbacks,
    verbose=1
)

# ============================================================
# 6) EVALUATION
# ============================================================
y_pred = model.predict({"num":Xn_test,"cat":Xc_test}, batch_size=1024)
def rmse(a,b): return math.sqrt(mean_squared_error(a,b))
def mape(a,b): return np.mean(np.abs((a - b) / np.clip(a, 1e-3, None))) * 100

results = {}
for h in range(H):
    results[f"RMSE_h{h+1}"] = rmse(y_test[:,h], y_pred[:,h])
    results[f"MAPE_h{h+1}"] = mape(y_test[:,h], y_pred[:,h])
results["RMSE_all"] = rmse(y_test.flatten(), y_pred.flatten())
results["MAPE_all"] = mape(y_test.flatten(), y_pred.flatten())
results


Epoch 1/30
2117/2117 ━━━━━━━━━━━━━━━━━━━━ 34s 12ms/step - loss: 21.4939 - mae: 3.3685 - val_loss: 18.1877 - val_mae: 3.1883 - learning_rate: 0.0010
Epoch 2/30
2117/2117 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 17.0304 - mae: 3.0486 - val_loss: 18.1567 - val_mae: 3.1612 - learning_rate: 0.0010
Epoch 3/30
2117/2117 ━━━━━━━━━━━━━━━━━━━━ 25s 12ms/step - loss: 16.8642 - mae: 3.0362 - val_loss: 18.2495 - val_mae: 3.1335 - learning_rate: 0.0010
Epoch 4/30
2117/2117 ━━━━━━━━━━━━━━━━━━━━ 25s 12ms/step - loss: 16.7391 - mae: 3.0266 - val_loss: 18.2176 - val_mae: 3.1714 - learning_rate: 0.0010
Epoch 5/30
2117/2117 ━━━━━━━━━━━━━━━━━━━━ 25s 12ms/step - loss: 16.5886 - mae: 3.0159 - val_loss: 18.2412 - val_mae: 3.1632 - learning_rate: 5.0000e-04
Epoch 6/30
2117/2117 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 16.5487 - mae: 3.0126 - val_loss: 18.2952 - val_mae: 3.1658 - learning_rate: 5.0000e-04
Epoch 7/30
2117/2117 ━━━━━━━━━━━━━━━━━━━━ 26s 12ms/step - loss: 16.5018 - mae: 3.0095 - val_loss: 18.288

{'RMSE_h1': 3.94787140000829,
 'MAPE_h1': np.float32(2969.3274),
 'RMSE_h2': 3.9509402785638046,
 'MAPE_h2': np.float32(2983.0798),
 'RMSE_h3': 3.959898803072954,
 'MAPE_h3': np.float32(2940.8853),
 'RMSE_h4': 3.9771044648483156,
 'MAPE_h4': np.float32(2974.5479),
 'RMSE_all': 3.95897052387963,
 'MAPE_all': np.float32(2966.9597)}

In [ ]:
configs = [
    {"conv_filters":32,"lstm":32,"dropout":0.2},
    {"conv_filters":64,"lstm":64,"dropout":0.25},
    {"conv_filters":96,"lstm":64,"dropout":0.3},
]

def build_model(cfg):
    inp_num = keras.Input(shape=(T, len(num_feats)), name="num")
    inp_cat = keras.Input(shape=(T, len(cat_feats)), name="cat", dtype="int32")

    idx = {name:i for i,name in enumerate(cat_feats)}
    cat_store = layers.Lambda(lambda z: z[:,:,idx["store_id"]])(inp_cat)
    cat_prod  = layers.Lambda(lambda z: z[:,:,idx["product_id"]])(inp_cat)
    cat_region= layers.Lambda(lambda z: z[:,:,idx["region_id"]])(inp_cat)
    cat_cat   = layers.Lambda(lambda z: z[:,:,idx["category_id"]])(inp_cat)
    cat_size  = layers.Lambda(lambda z: z[:,:,idx["store_size"]])(inp_cat)
    cat_promo = layers.Lambda(lambda z: z[:,:,idx["promo"]])(inp_cat)
    cat_hol   = layers.Lambda(lambda z: z[:,:,idx["is_holiday"]])(inp_cat)

    def emb(x,v,d): return layers.Embedding(v, d)(x)
    e = layers.Concatenate(axis=-1)([
        emb(cat_store, N_STORES, 8), emb(cat_prod, N_PRODUCTS, 8),
        emb(cat_region, N_REGIONS, 4), emb(cat_cat, N_CATEGORIES, 4),
        emb(cat_size, 4, 3), emb(cat_promo, 2, 2), emb(cat_hol, 2, 2)
    ])

    x = layers.Concatenate(axis=-1)([inp_num, e])
    x = layers.Conv1D(cfg["conv_filters"], 3, padding="causal", activation="relu")(x)
    x = layers.Bidirectional(layers.LSTM(cfg["lstm"], return_sequences=False))(x)
    x = layers.Dropout(cfg["dropout"])(x)
    x = layers.Dense(128, activation="relu")(x)
    out = layers.Dense(H, activation="relu")(x)
    m = keras.Model([inp_num, inp_cat], out)
    m.compile(optimizer=keras.optimizers.Adam(1e-3), loss="mse")
    return m

best, best_cfg = None, None
for cfg in configs:
    m = build_model(cfg)
    m.fit({"num":Xn_train,"cat":Xc_train}, y_train,
          validation_data=({"num":Xn_valid,"cat":Xc_valid}, y_valid),
          epochs=10, batch_size=512, verbose=0)
    pred = m.predict({"num":Xn_valid,"cat":Xc_valid}, verbose=0)
    score = mape(y_valid.flatten(), pred.flatten())
    print(cfg, "MAPE_valid:", round(score,2))
    if best is None or score < best:
        best, best_cfg = score, cfg

print("Best config:", best_cfg, "valid MAPE:", round(best,2))


{'conv_filters': 32, 'lstm': 32, 'dropout': 0.2} MAPE_valid: 2101.91
{'conv_filters': 64, 'lstm': 64, 'dropout': 0.25} MAPE_valid: 1515.87
{'conv_filters': 96, 'lstm': 64, 'dropout': 0.3} MAPE_valid: 2171.55
Best config: {'conv_filters': 64, 'lstm': 64, 'dropout': 0.25} valid MAPE: 1515.87


In [ ]:
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(resolver)
    with strategy.scope():
        model = ...  # (re)build the model here
        model.compile(optimizer=keras.optimizers.Adam(1e-3), loss="mse")
except Exception as e:
    print("TPU unavailable, running default:", e)


TPU unavailable, running default: Please provide a TPU Name to connect to.
